# Restore a Delta table

You can restore a Delta table to its earlier state by using the RESTORE command. A Delta table internally maintains historic versions of the table that enable it to be restored to an earlier state. A version corresponding to the earlier state or a timestamp of when the earlier state was created are supported as options by the RESTORE command.



In [ ]:
# Generate dummy data

from pyspark.sql.functions import *

data = [(1,"open"),(2,"close"),(3,"open"),(4,"open"),(5,"close")]
schema =["id","action"]

df = spark.createDataFrame(data=data, schema=schema)

delta_table_name = 'restore_demo'

spark.sql(f"DROP TABLE IF EXISTS {delta_table_name}")
df.write.format("delta").mode("overwrite").saveAsTable(delta_table_name)

## Version 0
Displaying versions (history) of delta tables

In [ ]:
import delta
from pyspark.sql.functions import *

delta_info = delta.DeltaTable.forName(spark, "demo.restore_demo")

display(delta_info.history())

    Checking some data before making changes

In [ ]:
%%sql

SELECT * 
FROM demo.restore_demo 

## Version 1

Now update some data to the delta table, which will create **version 1**

In [ ]:
delta_info.update(
    condition = col("id") == 4,
    set = {'action': "'close'"} 
    )

    OR

In [ ]:
%%sql

UPDATE demo.restore_demo SET action = 'close' where id = 4 

    Checking data after making changes

In [ ]:
%%sql

SELECT * 
FROM demo.restore_demo 

    Getting info of the new version   

# Restore by version

version is a value that can be obtained from the output of DESCRIBE HISTORY command (SQL) or history() method.

In [ ]:
delta_info = delta.DeltaTable.forName(spark, "demo.restore_demo")
display(delta_info.history())

In [ ]:
%%sql

DESCRIBE HISTORY demo.restore_demo

In [ ]:
delta_info.restoreToVersion(0)

In [ ]:
%%sql 

RESTORE TABLE covid TO VERSION AS OF 0

In [ ]:
%%sql

SELECT * 
FROM demo.restore_demo 

# Restore using timestamp

> Get the **timestamp for version 0** and _change_ the below commands

In [ ]:
delta_info = delta.DeltaTable.forName(spark, "demo.restore_demo")
display(delta_info.history())

In [ ]:
delta_info.restoreToTimestamp("2024-02-17 18:54:42.622")

In [ ]:
%%sql 

RESTORE TABLE demo.restore_demo TO TIMESTAMP AS OF '2024-02-17 18:45:24.192'

# Delta Log

Delta Lake log entries added by the RESTORE command contain dataChange set to true

In [ ]:
delta_info = delta.DeltaTable.forName(spark, "demo.restore_demo")
display(delta_info.history())

# Clean up

In [ ]:
spark.sql("DROP TABLE IF EXISTS demo.restore_demo")